*HAMMOUTI Douae*

This notebook is based on EPFL's Modern natural language processing course

## N-gram Language Models

In this exercise, we will better understand the functioning of different types of (non-neural) language modeling, namely,  Unigram LM, Bi-gram LM, and Tri-gram LM.

### Unigram <a name='unigram_lm'></a>
In the simple Unigram language model, we pick/generate next token independent of the previous token. In other words, during the generation, we pick the tokens according to the token probability. Therefore, for an arbitrary sequence $x_1x_2~...x_n$, its respective probability becomes:
$$p(x_1x_2~...x_n) = \Pi_{i=1} ^n p(x_i)$$
Let's use an unsupervised dataset (raw corpus) to evaluate this model's perplexity. We use Huggingface's `datasets` library to download the datasets.


Here we use the `Penn Treebank` dataset, featuring a million words of 1989 Wall Street Journal material. The rare words in this version are already replaced with `<unk>` token. The numbers are also replaced with a special token. This token replacement helps us to end up with a more reasonable vocabulary size to work with.

In [1]:
!pip install datasets==3.6.0

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.9 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 649.9 kB/s eta 0:00:00
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     ---------------------------------------- 76.3/76.3 kB 2.1 MB/s eta 0:00:00
Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
   ---------------------------------------- 0.0/116.3 


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\douae\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import datasets
from datasets import load_dataset_builder 

# downloading the dataset
builder = load_dataset_builder('ptb_text_only')
builder.download_and_prepare()
ptb_dataset = builder.as_dataset(split='train')

# splitting dataset in train/test (to be later used for language model evaluation)
ptb_dataset = ptb_dataset.train_test_split(test_size=0.2, seed=1)
ptb_train, ptb_test = ptb_dataset['train'], ptb_dataset['test']

C:\Users\douae\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's have a look at a few samples of the training dataset (and also the structure of the dataset)

In [3]:
for i in range(3):
    print(str(ptb_train[i])+'\n')

{'sentence': "a former executive agreed that the departures do n't reflect major problems adding if you see any company that grows as fast as reebok did it is going to have people coming and going"}

{'sentence': 'with talk today of a second economic <unk> in west germany east germany no longer can content itself with being the economic star in a loser league'}

{'sentence': 'transportation secretary sam skinner who earlier fueled the anti-takeover fires with his <unk> attacks on foreign investment in u.s. carriers now says the bill would further <unk> the jittery capital markets'}



During generation with a given language model, we often need to have a `<stop>` token in our vocabulary to terminate the generation of a given sentence/paragraph. In this dataset, every sample is a sentence, and the `<stop>` token should be added to the end of every sample (i.e., end of sentence).

We therefore need to create a new train/test dataset starting from `ptb_train` and `ptb_test` that has a `<stop>` at the end of each sentence.

In [4]:
from datasets import Dataset

def add_stop_token(input_sample: dict) -> dict:
    '''
    args:
        input_sample: a dict representing a sample of the dataset. (look above for the dict struture)
    output:
        modified_sample: modified dict adding <stop> at the end of each sentence.
    '''

    # YOUR CODE HERE
    # do not change the structure of the datasets objects, just change the respective sentences as discussed
    
    modified_sample = {
        'sentence': input_sample['sentence'].strip() + " <stop>"
    }
    return modified_sample

# apply here the function to the dataset
ptb_train = Dataset.from_list([add_stop_token(item) for item in ptb_train])

for i in range(3):
    print(str(ptb_train[i])+'\n')

assert(ptb_train[1]['sentence']=='with talk today of a second economic <unk> in west germany east germany no longer can content itself with being the economic star in a loser league <stop>')

{'sentence': "a former executive agreed that the departures do n't reflect major problems adding if you see any company that grows as fast as reebok did it is going to have people coming and going <stop>"}

{'sentence': 'with talk today of a second economic <unk> in west germany east germany no longer can content itself with being the economic star in a loser league <stop>'}

{'sentence': 'transportation secretary sam skinner who earlier fueled the anti-takeover fires with his <unk> attacks on foreign investment in u.s. carriers now says the bill would further <unk> the jittery capital markets <stop>'}



For the both `ptb_train` and `ptb_test` datasets, filter out every sample that has less than 3 tokens. it will help remove very short sentences that are not very helpful for training/evaluating a langugage model.

In [5]:
# YOUR CODE HERE

ptb_train = ptb_train.filter(lambda example: len(example['sentence'].split()) >= 3)
ptb_test = ptb_test.filter(lambda example: len(example['sentence'].split()) >= 3)


Filter: 100%|██████████| 8414/8414 [00:00<00:00, 84997.49 examples/s]


Now let's create a dictionary of the word probabilites (in the format of `{word: Prob(word)}`in the following function. We will use these probabilities to estimate sequence probabilities for a given sequence, as mentioned above.

In [6]:
from collections import defaultdict, Counter
from collections import Counter

def get_word_probability_dict(train_dataset: datasets.arrow_dataset.Dataset):
    '''
    args:
        train_dataset: a Dataset object that can be iterated to get all the sentences
    output:
        word_prob_dict: a dictionary containing the word probabilities
    '''

    # YOUR CODE HERE
    # tokenize and compute the probability
    word_count = Counter()
    total_words = 0  
    for item in train_dataset:
        words = item['sentence'].split()
        word_count.update(words)
        total_words += len(words)  

    word_prob_dict = {word: count / total_words for word, count in word_count.items()}  
    return word_prob_dict

word_prob_dict = get_word_probability_dict(ptb_train)

In [7]:
word_prob_dict

{'a': 0.022753647772832404,
 'former': 0.00033522621711350035,
 'executive': 0.0006354488934842256,
 'agreed': 0.00032310960685638586,
 'that': 0.009558659202834748,
 'the': 0.054680915800329036,
 'departures': 1.211661025711447e-05,
 'do': 0.0010878023430831657,
 "n't": 0.0036269053369629312,
 'reflect': 5.78904712284358e-05,
 'major': 0.0007000708148555026,
 'problems': 0.00032445589688495413,
 'adding': 8.346998177123302e-05,
 'if': 0.0013853324393967544,
 'you': 0.0008468164279694446,
 'see': 0.00030291525642786176,
 'any': 0.0008885514188550611,
 'company': 0.0029281808121359968,
 'grows': 1.7501770371387566e-05,
 'as': 0.00519667951027354,
 'fast': 4.173499088561651e-05,
 'reebok': 2.2886930485660663e-05,
 'did': 0.0006556432439127496,
 'it': 0.006556432439127497,
 'is': 0.007975422129238458,
 'going': 0.00035542056754202445,
 'to': 0.025439496379826114,
 'have': 0.0034424636030490775,
 'people': 0.0008791273886550832,
 'coming': 0.0001575159333424881,
 'and': 0.01886556217032723

Let's also get a sense of how high the top-k probabilities are:

In [8]:
import operator

sorted(word_prob_dict.items(), key=operator.itemgetter(1), reverse=True)[:20]

[('the', 0.054680915800329036),
 ('<unk>', 0.04831565654525823),
 ('<stop>', 0.045153221268151356),
 ('N', 0.03495776688180381),
 ('of', 0.026197457665910053),
 ('to', 0.025439496379826114),
 ('a', 0.022753647772832404),
 ('in', 0.019484855583468637),
 ('and', 0.01886556217032723),
 ("'s", 0.010568376724260954),
 ('for', 0.009569429523063295),
 ('that', 0.009558659202834748),
 ('$', 0.00801984970018121),
 ('is', 0.007975422129238458),
 ('it', 0.006556432439127497),
 ('said', 0.006516043738270448),
 ('on', 0.006050227388385825),
 ('at', 0.005312460452730411),
 ('by', 0.005303036422530433),
 ('as', 0.00519667951027354)]

The probability of 'the' should be ~ 0.05468

for 'and' instead should be ~ 0.01886

Now let's analyze the Unigram language model for different sequences. We first create a function that can output the probability for a given string (probability of the entire sequence).

In [9]:
def unigram_lm_seq_probability(input_sentence: str,
                               word_prob_dict: dict):
    '''
    args:
        input_sentence: The input sequence string.
        word_prob_dict: A dictionary containing the probability for a given token
    output:
        probability: The probability of the input_sentence according to the Unigram language model
    '''

    # YOUR CODE HERE
    words = input_sentence.split()
    probability = 1.0
    for word in words:
        word_probability = word_prob_dict.get(word, 1e-6)  # Use a small probability for unknown words
        probability *= word_probability 

    probability *= (1e-6)  # Apply a small probability for the end of the sentence
    return probability

Let's investigate a major issue with Unigram language model

In [10]:
seq1 = "the the the the <stop>"
seq2 = "i love computer science <stop>"

prob_seq1 = unigram_lm_seq_probability(seq1, word_prob_dict)
prob_seq2 = unigram_lm_seq_probability(seq2, word_prob_dict)
print(f"probability for seq1 is {prob_seq1}, and for seq2 is {prob_seq2}")

probability for seq1 is 4.0367500274713214e-13, and for seq2 is 2.3485816949091946e-23


the probability for seq1 should be ~ 4e-07 and for seq2 ~ 2e-17

**How can we avoid having large probability values for sequences like `the the the <stop>`? What caused this behavior?**

*Here your answer*

The problem happens because unigram assumes independence and "the" is very frequent.

We fix it by using context-aware models (bigrams/trigrams/neural LMs) or by applying smoothing.



Now let's formally evaluate the Unigram model in terms of perplexity. We first compute the entropy as the average negative log-likelihood:
$$H(W_{test})= \frac{1}{|W_{test}|} \sum_{w\in W_{test}} −log_2P(w)$$
where $W_{test}$ is the input sequence (words)

Note that the logarithm is in base 2.

In order to get a reliable value, we will do the above calculation for all the sentences in `ptb_test` dataset and then an average is taken over all these samples.

In [11]:
def get_unigram_lm_entropy(input_sentence: str,
                           word_prob_dict: dict):
    '''
    args:
        input_sentence: the input string that we would like to have its respective entropy value.
        word_prob_dict: A dictionary containing the probability for a given token
    output:
        entropy: entropy value as defined above
    '''

    # YOUR CODE HERE
    # Hints:
    # 1) use numpy to compute mean and logarithm
    # 2) use get() function to get values from dictionary to avoid problem with non existing keys
    # e.g. you can use word_prob_dict.get(word)==None to check if the key does not exist
    words = input_sentence.split()
    probabilities = [word_prob_dict.get(word, 1e-6) for word in words]  # Use a small probability for unknown words
    log_probabilities = np.log2(probabilities)
    entropy = -np.mean(log_probabilities)   


    return entropy

unigram_lm_entropy = [get_unigram_lm_entropy(i["sentence"], word_prob_dict) for i in ptb_test]

In [12]:
unigram_lm_entropy[:500]

[np.float64(8.106856549838785),
 np.float64(8.45689624015916),
 np.float64(9.2255972226014),
 np.float64(10.703441841833799),
 np.float64(9.910421865968472),
 np.float64(10.047989658727857),
 np.float64(9.486811988069084),
 np.float64(9.533671238708457),
 np.float64(7.4848763484131915),
 np.float64(9.471955873966571),
 np.float64(10.616011340139918),
 np.float64(10.567008455447766),
 np.float64(7.689351657476617),
 np.float64(9.11216216311631),
 np.float64(8.989010244609007),
 np.float64(10.087269167731876),
 np.float64(11.765551932426632),
 np.float64(8.943194739420157),
 np.float64(10.821023665275636),
 np.float64(10.035795075015145),
 np.float64(8.684025546579644),
 np.float64(10.098586691543469),
 np.float64(10.107930137117538),
 np.float64(9.50963284070317),
 np.float64(8.54819408610428),
 np.float64(10.318216011607717),
 np.float64(9.332191554448963),
 np.float64(8.77809488988812),
 np.float64(10.615157041890384),
 np.float64(9.842332667981411),
 np.float64(11.321507337822664),
 

**Why this method could generate infinitesimal values? What caused this phenomenon?**

*Here your answer*

Probabilities are small, and multiplying many small numbers makes them infinitesimal.

It happens because of long sequences and the independence assumption.

We avoid it by using logs instead of raw products.

Now compute entropy for all the sentences in the `ptb_test` given above function and then compute the average entropy. Finally, compute the perplexity as $2^{\bar{H}}$, where $\bar{H}$ is the average entropy over the test dataset.

In [13]:
def get_unigram_lm_perplexity(test_dataset: datasets.arrow_dataset.Dataset,
                              word_prob_dict: dict):
    '''
    args:
        test_dataset: the test dataset samples are used to compute the perplexity for the Unigram LM.
        word_prob_dict: A dictionary containing the probability for a given token
    output:
        perplexity: entropy value as defined above
    '''

    # YOUR CODE HERE
    # keep in mind to filter out infinite entropy values
    # Hint: use np.isinf()
    entropies = [get_unigram_lm_entropy(item['sentence'], word_prob_dict) for item in test_dataset]
    finite_entropies = [e for e in entropies if not np.isinf(e)]
    mean_entropy = np.mean(finite_entropies)
    perplexity = 2 ** mean_entropy  


    return perplexity

unigram_lm_perplexity = get_unigram_lm_perplexity(ptb_test, word_prob_dict)

In [14]:
print(f"The perplexity for the Unigram language model is {unigram_lm_perplexity}")

The perplexity for the Unigram language model is 841.072820189752


the perplexity should be ~ 840

**We trained this model on the Penn Treebank dataset, could we compare this perplexity value with models trained on other datasets too? Explain your answer.**

*Here you answer*

You can’t directly compare PP across datasets because it depends on the dataset’s words and style.

You can compare models only if they are evaluated on the same data.

## Bi-gram Language Model <a name='bigram_lm'></a>
In the Bi-gram language model, we pick/generate next token conditioned on the previous token. Therefore, for an arbitrary sequence $x_1x_2~...x_n$, its respective probability becomes:
$$p(x_1x_2~...x_n) = p(x_1) ~\Pi_{i=2} ^n p(x_i|x_{i-1})$$
Let's use the same dataset (`Penn Treebank`) to evaluate this model's perplexity. (We use the dataset that already has the `<stop>` token at the end).

We estimate $p(x_i|x_{i-1})$ as the $\frac{count(x_{i-1},~x_i)}{count(x_{i-1})}$ according to the training dataset frequencies.

In [15]:
def get_first_order_conditional_probabilities(train_dataset: datasets.arrow_dataset.Dataset):
    '''
    In this function the conditional probabilities have to be computed based on train_dataset. The output of the
    function is a dictionary having keys like (x_{i-1}, x_i) as a tuple and the value being p(x_i|x_{i-1}).
    args:
        train_dataset: a Dataset object that can be iterated to get all the sentences
    output:
        first_order_condition_prob: a dictionary containing the first order conditional probabilities
    '''

    first_order_condition_prob = defaultdict(float) # in order to get zeroes
    word_prob_dict = get_word_probability_dict(train_dataset) # let's first get the word frequencies (later used for computation of conditional probabilities)

    # YOUR CODE here
    # generate all the bigrams on the training set, compute their frequencies
    # then compute conditional probabilities for each bigram by dividing its frequency by the frequency of the first word in the bigram
    bigram_count = Counter()
    unigram_count = Counter()
    for item in train_dataset:  
        words = item['sentence'].split()
        bigrams = zip(words[:-1], words[1:])
        bigram_count.update(bigrams)
        unigram_count.update(words[:-1])  # Count occurrences of the first word in each bigram          

    for (w1, w2), bigram_freq in bigram_count.items():
        if unigram_count[w1] > 0:
            first_order_condition_prob[(w1, w2)] = bigram_freq / unigram_count[w1]
        else:
            first_order_condition_prob[(w1, w2)] = 0.0  # Avoid division by zero

    return first_order_condition_prob

first_order_condition_prob = get_first_order_conditional_probabilities(ptb_train)

In [16]:
first_order_condition_prob

defaultdict(float,
            {('a', 'former'): 0.0038459262765516834,
             ('former', 'executive'): 0.01606425702811245,
             ('executive', 'agreed'): 0.00211864406779661,
             ('agreed', 'that'): 0.04583333333333333,
             ('that', 'the'): 0.14,
             ('the', 'departures'): 9.848335631278314e-05,
             ('departures', 'do'): 0.1111111111111111,
             ('do', "n't"): 0.504950495049505,
             ("n't", 'reflect'): 0.001855976243504083,
             ('reflect', 'major'): 0.023255813953488372,
             ('major', 'problems'): 0.009615384615384616,
             ('problems', 'adding'): 0.004149377593360996,
             ('adding', 'if'): 0.016129032258064516,
             ('if', 'you'): 0.08843537414965986,
             ('you', 'see'): 0.009538950715421303,
             ('see', 'any'): 0.022222222222222223,
             ('any', 'company'): 0.0015151515151515152,
             ('company', 'that'): 0.016551724137931035,
             (

Now let's analyze the Bi-gram language model for different sequences. We first create a function that can output the probability for a given string.

In [22]:
def bigram_lm_seq_probability(input_sentence: str,
                              word_prob_dict: dict,
                              first_order_condition_prob: dict):
    '''
    args:
        input_sentence: The input sequence string
        word_prob_dict: a dictionary containing the word probabilities
        first_order_condition_prob: a dictionary containing the first order conditional probabilities
    output:
        probability: The probability of the input_sentence according to the Bi-gram language model
    '''

    # YOUR CODE HERE
    # compute the probability for the entire sequence
    # Hint: generate all the bigrams and rely on the dictionary we created with the previous function
    # for the first token of the sequence (a unigram instead of a bigram) just use the unigram probability (word probabilities)
    words = input_sentence.split()
    probability = word_prob_dict.get(words[0], 0)  # Start with the
    for i in range(1, len(words)):
        bigram = (words[i-1], words[i])
        bigram_probability = first_order_condition_prob.get(bigram, 0)  # Get the conditional probability of the bigram
        probability *= bigram_probability  # Multiply the probabilities

    return probability

Let's investigate a major issue with higher order language models.

In [23]:
bigram_lm_seq_probability('the panda',word_prob_dict,first_order_condition_prob)

0.0

Compute the probabilities for all the sequences in `ptb_test` dataset

In [24]:
bigram_test_probabilities = [bigram_lm_seq_probability(sample["sentence"],word_prob_dict,first_order_condition_prob) for sample in ptb_test]

How many zero probability do we have?

In [25]:
# YOUR CODE HERE
# compute the % of zero probabilities in the test set

# YOUR CODE HERE
# compute the % of zero probabilities in the test set
zero_prob_count = sum(1 for prob in bigram_test_probabilities if prob == 0)
total_count = len(bigram_test_probabilities)
zero_prob_percentage = (zero_prob_count / total_count) * 100 if total_count > 0 else 0
print(f"Percentage of zero probabilities in the test set: {zero_prob_percentage:.2f}%")




Percentage of zero probabilities in the test set: 91.19%


It should be ~91%

**What caused this number of zero probabilities? What is this phenomenon called and how it can be overcome?**

*Here your answer*

Zero probabilities happen because the model never saw some words.

This is called the zero-frequency problem.

Smoothing gives unseen words a tiny probability so calculations don’t break.

Now compute the Bigram language model perplexity over `ptb_test` dataset over all the sentences.

In [27]:
# YOUR CODE HERE
# compute the average perplexity over the test set
# you can avoid having two functions (one for entropy and one for perplexity)
# Hint: skip samples (sentences) with zero probability

def get_bigram_lm_perplexity(test_dataset: datasets.arrow_dataset.Dataset,      
                                word_prob_dict: dict,
                                first_order_condition_prob: dict):
        '''
        args:
            test_dataset: the test dataset samples are used to compute the perplexity for the Bi-gram LM.
            word_prob_dict: A dictionary containing the probability for a given token
            first_order_condition_prob: a dictionary containing the first order conditional probabilities
        output:
            perplexity: entropy value as defined above
        '''
    
        # YOUR CODE HERE
        # keep in mind to filter out infinite entropy values
        # Hint: use np.isinf()
        entropies = []
        for item in test_dataset:
            prob = bigram_lm_seq_probability(item['sentence'], word_prob_dict, first_order_condition_prob)
            if prob > 0:
                entropy = -np.log2(prob) / len(item['sentence'].split())  # Normalize by sentence length
                entropies.append(entropy)
    
        finite_entropies = [e for e in entropies if not np.isinf(e)]
        mean_entropy = np.mean(finite_entropies) if finite_entropies else float('inf')
        perplexity = 2 ** mean_entropy if mean_entropy != float('inf') else float('inf')
    
        return perplexity
bigram_lm_perplexity = get_bigram_lm_perplexity(ptb_test, word_prob_dict, first_order_condition_prob)

print(f"Bigram language model perplexity is {bigram_lm_perplexity}")


Bigram language model perplexity is 49.818733285966566


The perplexity should be ~ 49

**Why the perplexity of the bigram model is so low compared to the unigram model?**


*Here your answer*

The bigram model “remembers” the previous word, so it can predict the next word better than the unigram model, making the overall sequence more likely and the perplexity smaller.

## +++ End of the mandatory section +++

## **Advance only**

Fix the problem of exorbitant number of zero probabilities in the Bi-gram language model by adding some smoothing to the probabilities. In this exercise, use Laplace smoothing as defined below:
$$P(x_i|x_{i-1}) = \frac{count(x_{i-1},~x_i) + \alpha}{count(x_{i-1}) + \alpha ~|V|}$$
where $\alpha$ is the smoothing parameter, and $|V|$ is the (train dataset) vocabulary size.
Implement the Laplace smoothing and recompute the conditional probabilities with $\alpha=3\cdot10^{-3}$.

Finally, define sequence probability of the Tri-gram language model.
In the Tri-gram language model, we pick/generate next token conditioned only on the previous token. Therefore, for an arbitrary sequence $x_1x_2~...x_n$, its respective probability becomes:
$$p(x_1x_2~...x_n) = p(x_1) p(x_2|x_1) ~\Pi_{i=3} ^n p(x_i|x_{i-2}x_{i-1})$$
Compute the probability for all the sequences in the test dataset and  compute perplexity of this model.